# E5 — Martin Dimension Test

## Goal
Benchmark $\dim S_k^{\text{new}}(\Gamma_0(N))$ for semiprimes $N = pq$ up to ~50,000.  
Fit $T(N) \propto N^\alpha$ and compare against trial-division / ECM baselines.

## Key question
Does computing the dimension of the newform subspace scale as $N^\alpha$ with $\alpha < 2$?  
If $\alpha < 1$: genuinely surprising — this path has real potential.  
If $\alpha \geq 2$: this path is theory-only unless a trace-formula shortcut appears.

## Method
1. Generate semiprimes $N = pq$ with $p < q$ across a range of sizes.
2. For each $N$, time `ModularSymbols(Gamma0(N), k, sign=1).cuspidal_subspace().new_subspace().dimension()` for $k=2$.
3. Also time the dimension formula (Martin's formula) which uses divisor arithmetic.
4. Fit power-law $T(N) = c \cdot N^\alpha$.
5. Compare to trial division and ECM timings on the same $N$ values.

In [ ]:
import time
import numpy as np
from sage.all import *
import json
import os

## 1. Generate semiprimes $N = pq$ across a range of magnitudes

In [ ]:
def generate_semiprimes(max_N, num_samples_per_decade=15):
    """
    Generate semiprimes N = p*q with p < q, distributed
    roughly log-uniformly across [100, max_N].
    
    We sample across decades to get good coverage for power-law fitting.
    """
    semiprimes = []
    
    # Log-spaced target values
    log_min, log_max = np.log10(100), np.log10(max_N)
    targets = np.logspace(log_min, log_max, num_samples_per_decade * int(log_max - log_min + 1))
    
    for target in targets:
        N_approx = int(target)
        if N_approx < 6:
            continue
        # Find a semiprime near the target
        # Try p near sqrt(N) for balanced, and also some unbalanced ones
        found = False
        for offset in range(200):
            candidate = N_approx + offset
            if candidate < 6:
                continue
            f = factor(candidate)
            if len(f) == 2 and all(e == 1 for _, e in f):
                p, q = f[0][0], f[1][0]
                semiprimes.append((int(candidate), int(p), int(q)))
                found = True
                break
        if not found:
            # fallback: construct one
            p = next_prime(isqrt(N_approx))
            q = next_prime(p + 1)
            semiprimes.append((int(p*q), int(p), int(q)))
    
    # Deduplicate and sort
    seen = set()
    unique = []
    for N, p, q in semiprimes:
        if N not in seen:
            seen.add(N)
            unique.append((N, p, q))
    unique.sort()
    return unique

semiprimes = generate_semiprimes(50000, num_samples_per_decade=20)
print(f"Generated {len(semiprimes)} semiprimes from {semiprimes[0][0]} to {semiprimes[-1][0]}")
print(f"First few: {semiprimes[:5]}")
print(f"Last few:  {semiprimes[-5:]}")

## 2. Benchmark: Modular Symbols computation of $\dim S_k^{\text{new}}(\Gamma_0(N))$

We use SageMath's `ModularSymbols` to compute the cuspidal new subspace dimension.  
This exercises the modular-symbols algorithm — the question is what power of $N$ it runs in.

In [ ]:
def benchmark_newform_dim_modsym(N, k=2, num_trials=1):
    """
    Compute dim S_k^new(Gamma_0(N)) via modular symbols.
    Returns (dimension, elapsed_seconds).
    
    We time the full computation: constructing the space,
    computing the cuspidal subspace, then the new subspace.
    """
    times = []
    dim = None
    for _ in range(num_trials):
        t0 = time.perf_counter()
        M = ModularSymbols(Gamma0(N), k, sign=1)
        S = M.cuspidal_subspace()
        Snew = S.new_subspace()
        dim = Snew.dimension()
        t1 = time.perf_counter()
        times.append(t1 - t0)
    return dim, min(times)  # use min to reduce noise

# Quick sanity check
test_N = 77  # = 7 * 11
d, t = benchmark_newform_dim_modsym(test_N)
print(f"dim S_2^new(Gamma_0({test_N})) = {d}, time = {t:.4f}s")

## 3. Benchmark: Dimension formula (analytic / divisor-based)

The new subspace dimension is obtained by Möbius-inverting the old-form decomposition:
$$\dim S_k(\Gamma_0(N)) = \sum_{M | N} \sigma_0(N/M) \, \dim S_k^{\text{new}}(\Gamma_0(M))$$

Inverting gives $\dim S_k^{\text{new}}(\Gamma_0(N)) = \sum_{d | N} (\mu * \mu)(N/d) \, \dim S_k(\Gamma_0(d))$,
where $(\mu * \mu)$ is the Dirichlet convolution of Möbius with itself.

For $N = pq$ squarefree: divisors $\{1, p, q, pq\}$, and
$(\mu*\mu)(1) = 1$, $(\mu*\mu)(p) = -2$, $(\mu*\mu)(q) = -2$, $(\mu*\mu)(pq) = 4$.

This requires iterating over **divisors of $N$** — computing it requires *knowing* $p$ and $q$.
We benchmark this to understand the *formula's* computation time separately from modular symbols.

In [ ]:
def dim_Sk_Gamma0(N, k=2):
    """
    Classical dimension formula for dim S_k(Gamma_0(N)) for k >= 2 even.
    Uses the standard formula involving genus, elliptic points, and cusps.
    """
    if k < 2 or k % 2 != 0:
        raise ValueError("k must be even and >= 2")
    if k == 2:
        return Gamma0(N).genus()
    # For general even k >= 4:
    return dimension_cusp_forms(Gamma0(N), k)

def mu_star_mu(n):
    """
    Dirichlet convolution (mu * mu)(n) = sum_{d|n} mu(d) * mu(n/d).
    This is the Dirichlet inverse of sigma_0 (the divisor-count function).
    
    For squarefree n = p1*p2*...*pk:
        (mu * mu)(n) = (-2)^k  if n is squarefree with k prime factors
    For n with a prime-square factor p^2: (mu * mu)(n) = 0 if any exponent >= 3,
        and for p^2 exactly: (mu * mu)(p^2) = 1.
    """
    divs = divisors(n)
    return sum(moebius(d) * moebius(n // d) for d in divs)

def dim_Sk_new_formula(N, k=2):
    """
    Compute dim S_k^new(Gamma_0(N)) via inversion of the old-form decomposition.
    
    dim S_k(Gamma_0(N)) = sum_{M|N} sigma_0(N/M) * dim S_k^new(Gamma_0(M))
    
    Inverting: dim S_k^new(Gamma_0(N)) = sum_{d|N} (mu*mu)(N/d) * dim S_k(Gamma_0(d))
    """
    divs = divisors(N)
    result = sum(mu_star_mu(N // d) * dim_Sk_Gamma0(d, k) for d in divs)
    return result

def benchmark_newform_dim_formula(N, k=2, num_trials=3):
    """
    Time the dimension formula computation.
    """
    times = []
    dim = None
    for _ in range(num_trials):
        t0 = time.perf_counter()
        dim = dim_Sk_new_formula(N, k)
        t1 = time.perf_counter()
        times.append(t1 - t0)
    return dim, min(times)

# Sanity check: should agree with modular symbols
d_formula, t_formula = benchmark_newform_dim_formula(test_N)
print(f"Formula: dim S_2^new(Gamma_0({test_N})) = {d_formula}, time = {t_formula:.6f}s")
assert d_formula == d, f"Mismatch: formula gives {d_formula}, modsym gives {d}"

## 4. Benchmark: Trial division and ECM baselines

In [ ]:
def benchmark_trial_division(N, num_trials=3):
    """Time naive trial division up to sqrt(N)."""
    times = []
    result = None
    for _ in range(num_trials):
        t0 = time.perf_counter()
        n = int(N)
        if n % 2 == 0:
            result = 2
        else:
            i = 3
            while i * i <= n:
                if n % i == 0:
                    result = i
                    break
                i += 2
            else:
                result = n  # prime
        t1 = time.perf_counter()
        times.append(t1 - t0)
    return result, min(times)

def benchmark_ecm(N, num_trials=1):
    """Time SageMath's ECM factorization."""
    times = []
    result = None
    for _ in range(num_trials):
        t0 = time.perf_counter()
        result = ecm.factor(N)
        t1 = time.perf_counter()
        times.append(t1 - t0)
    return result, min(times)

# Quick test
td_res, td_t = benchmark_trial_division(test_N)
print(f"Trial division of {test_N}: factor={td_res}, time={td_t:.6f}s")

ecm_res, ecm_t = benchmark_ecm(test_N)
print(f"ECM of {test_N}: factors={ecm_res}, time={ecm_t:.6f}s")

## 5. Run the full benchmark suite

In [ ]:
results = []

print(f"{'N':>8} {'p':>6} {'q':>6} {'dim':>5} {'modsym_t':>10} {'formula_t':>10} {'td_t':>10} {'ecm_t':>10}")
print("-" * 78)

for i, (N, p, q) in enumerate(semiprimes):
    try:
        # Modular symbols benchmark
        dim_ms, t_ms = benchmark_newform_dim_modsym(N, k=2, num_trials=1)
        
        # Dimension formula benchmark
        dim_f, t_f = benchmark_newform_dim_formula(N, k=2, num_trials=3)
        
        # Trial division benchmark
        _, t_td = benchmark_trial_division(N, num_trials=3)
        
        # ECM benchmark
        _, t_ecm = benchmark_ecm(N, num_trials=1)
        
        # Verify consistency
        assert dim_ms == dim_f, f"Dimension mismatch at N={N}: modsym={dim_ms}, formula={dim_f}"
        
        row = {
            'N': int(N), 'p': int(p), 'q': int(q),
            'dim_new': int(dim_ms),
            'time_modsym': float(t_ms),
            'time_formula': float(t_f),
            'time_trial_div': float(t_td),
            'time_ecm': float(t_ecm),
            'log_N': float(np.log10(N)),
            'log_time_modsym': float(np.log10(t_ms)) if t_ms > 0 else None,
        }
        results.append(row)
        
        print(f"{N:>8} {p:>6} {q:>6} {dim_ms:>5} {t_ms:>10.4f} {t_f:>10.6f} {t_td:>10.6f} {t_ecm:>10.6f}")
        
    except Exception as e:
        print(f"  ERROR at N={N}: {e}")
        continue

print(f"\nCompleted {len(results)} / {len(semiprimes)} benchmarks")

In [ ]:
# Save raw results
output_path = os.path.join('..', 'data', 'E5_benchmark_results.json')
with open(output_path, 'w') as f:
    json.dump(results, f, indent=2)
print(f"Saved {len(results)} results to {output_path}")

## 6. Power-law fitting: $T(N) = c \cdot N^\alpha$

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy import stats

# Extract arrays
Ns = np.array([r['N'] for r in results])
t_modsym = np.array([r['time_modsym'] for r in results])
t_formula = np.array([r['time_formula'] for r in results])
t_td = np.array([r['time_trial_div'] for r in results])
t_ecm = np.array([r['time_ecm'] for r in results])

# Power-law fit: log T = alpha * log N + log c
# Use only positive times for log fitting
mask = t_modsym > 0
log_N = np.log10(Ns[mask])
log_t = np.log10(t_modsym[mask])

slope, intercept, r_value, p_value, std_err = stats.linregress(log_N, log_t)
alpha_modsym = slope
c_modsym = 10**intercept

print(f"=== Modular Symbols Power-Law Fit ===")
print(f"  T(N) = {c_modsym:.2e} * N^{alpha_modsym:.3f}")
print(f"  alpha = {alpha_modsym:.4f} ± {std_err:.4f}")
print(f"  R² = {r_value**2:.6f}")
print()

# Also fit trial division for comparison
mask_td = t_td > 0
if mask_td.sum() > 2:
    slope_td, intercept_td, r_td, _, se_td = stats.linregress(
        np.log10(Ns[mask_td]), np.log10(t_td[mask_td]))
    print(f"=== Trial Division Power-Law Fit ===")
    print(f"  T(N) = {10**intercept_td:.2e} * N^{slope_td:.3f}")
    print(f"  alpha = {slope_td:.4f} ± {se_td:.4f}")
    print(f"  R² = {r_td**2:.6f}")

# Fit ECM
mask_ecm = t_ecm > 0
if mask_ecm.sum() > 2:
    slope_ecm, intercept_ecm, r_ecm, _, se_ecm = stats.linregress(
        np.log10(Ns[mask_ecm]), np.log10(t_ecm[mask_ecm]))
    print(f"\n=== ECM Power-Law Fit ===")
    print(f"  T(N) = {10**intercept_ecm:.2e} * N^{slope_ecm:.3f}")
    print(f"  alpha = {slope_ecm:.4f} ± {se_ecm:.4f}")
    print(f"  R² = {r_ecm**2:.6f}")

In [ ]:
# Also try piecewise fit to detect if alpha changes with scale
# Split at median N
median_N = np.median(Ns)

mask_lo = (Ns < median_N) & (t_modsym > 0)
mask_hi = (Ns >= median_N) & (t_modsym > 0)

if mask_lo.sum() > 2 and mask_hi.sum() > 2:
    sl_lo, int_lo, r_lo, _, se_lo = stats.linregress(
        np.log10(Ns[mask_lo]), np.log10(t_modsym[mask_lo]))
    sl_hi, int_hi, r_hi, _, se_hi = stats.linregress(
        np.log10(Ns[mask_hi]), np.log10(t_modsym[mask_hi]))
    
    print(f"=== Piecewise Fit (detect scaling change) ===")
    print(f"  N < {median_N:.0f}: alpha = {sl_lo:.4f} ± {se_lo:.4f}, R² = {r_lo**2:.4f}")
    print(f"  N >= {median_N:.0f}: alpha = {sl_hi:.4f} ± {se_hi:.4f}, R² = {r_hi**2:.4f}")
    
    if abs(sl_hi - sl_lo) > 2 * max(se_lo, se_hi):
        print(f"  ** Significant change in scaling exponent detected! **")
    else:
        print(f"  Scaling exponent appears stable across the range.")

## 7. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# --- Plot 1: Log-log timing comparison ---
ax = axes[0, 0]
ax.scatter(Ns, t_modsym, s=15, alpha=0.7, label='Modular Symbols', color='blue')
ax.scatter(Ns, t_formula, s=15, alpha=0.7, label='Dimension Formula', color='green')
ax.scatter(Ns, t_td, s=15, alpha=0.7, label='Trial Division', color='orange')
ax.scatter(Ns, t_ecm, s=15, alpha=0.7, label='ECM', color='red')

# Overlay fit line for modular symbols
N_fit = np.logspace(np.log10(Ns.min()), np.log10(Ns.max()), 100)
ax.plot(N_fit, c_modsym * N_fit**alpha_modsym, 'b--', lw=1.5,
        label=f'ModSym fit: $N^{{{alpha_modsym:.2f}}}$')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('N (semiprime)')
ax.set_ylabel('Time (seconds)')
ax.set_title('E5: Timing Comparison (log-log)')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# --- Plot 2: Modular symbols with fit residuals ---
ax = axes[0, 1]
predicted_log_t = alpha_modsym * np.log10(Ns[mask]) + np.log10(c_modsym)
residuals = np.log10(t_modsym[mask]) - predicted_log_t
ax.scatter(Ns[mask], residuals, s=15, alpha=0.7, color='blue')
ax.axhline(0, color='black', lw=0.5)
ax.set_xscale('log')
ax.set_xlabel('N')
ax.set_ylabel('log₁₀(T) residual')
ax.set_title(f'Fit Residuals (α={alpha_modsym:.3f})')
ax.grid(True, alpha=0.3)

# --- Plot 3: Dimension vs N ---
ax = axes[1, 0]
dims = np.array([r['dim_new'] for r in results])
ax.scatter(Ns, dims, s=15, alpha=0.7, color='purple')
ax.set_xscale('log')
ax.set_xlabel('N (semiprime)')
ax.set_ylabel('dim $S_2^{new}(\Gamma_0(N))$')
ax.set_title('Newform Dimension vs N')
ax.grid(True, alpha=0.3)

# --- Plot 4: Speedup ratio ---
ax = axes[1, 1]
ratio_td = t_td / t_modsym
ratio_ecm = t_ecm / t_modsym
valid = (t_modsym > 0) & (t_td > 0)
ax.scatter(Ns[valid], ratio_td[valid], s=15, alpha=0.7, label='TrialDiv / ModSym', color='orange')
valid_ecm = (t_modsym > 0) & (t_ecm > 0)
ax.scatter(Ns[valid_ecm], ratio_ecm[valid_ecm], s=15, alpha=0.7, label='ECM / ModSym', color='red')
ax.axhline(1, color='black', lw=0.5, ls='--', label='Break-even')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('N (semiprime)')
ax.set_ylabel('Time ratio (factoring / modular symbols)')
ax.set_title('Relative Speed: Factoring vs ModSym')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join('..', 'data', 'E5_timing_plots.png'), dpi=150)
plt.show()
print("Saved plots to data/E5_timing_plots.png")

## 8. Summary and Verdict

In [ ]:
print("=" * 70)
print("E5 MARTIN DIMENSION TEST — SUMMARY")
print("=" * 70)
print()
print(f"Tested {len(results)} semiprimes N = pq in [{results[0]['N']}, {results[-1]['N']}]")
print()
print(f"Modular Symbols scaling: T(N) ∝ N^{alpha_modsym:.3f} (R²={r_value**2:.4f})")
print()

if alpha_modsym < 1.0:
    verdict = "SURPRISING — sublinear in N! This path has genuine potential."
elif alpha_modsym < 2.0:
    verdict = "INTERESTING — superlinear but sub-quadratic. Worth investigating trace-formula shortcuts."
elif alpha_modsym < 3.0:
    verdict = "EXPECTED — quadratic to cubic. Modular symbols are polynomial in N (exponential in log N). Theory-only unless shortcut found."
else:
    verdict = "STEEP — cubic or worse. This path is very unlikely to yield a practical speedup."

print(f"VERDICT: {verdict}")
print()
print("Comparison at largest N tested:")
last = results[-1]
print(f"  N = {last['N']}: ModSym = {last['time_modsym']:.4f}s, "
      f"TrialDiv = {last['time_trial_div']:.6f}s, ECM = {last['time_ecm']:.6f}s")
print(f"  ModSym is {last['time_modsym']/max(last['time_trial_div'], 1e-10):.0f}x slower than trial division")
print(f"  ModSym is {last['time_modsym']/max(last['time_ecm'], 1e-10):.0f}x slower than ECM")
print()
print("NOTE: For factoring, what matters is scaling in log(N) = number of digits.")
print(f"      ModSym alpha in terms of N means alpha*log(N) in terms of digits.")
print(f"      At current alpha={alpha_modsym:.2f}, the modular-symbols approach")
print(f"      is O(N^{alpha_modsym:.2f}) = O(exp({alpha_modsym:.2f} * ln(N))) in the number size.")
print(f"      Trial division is O(N^0.5) = O(exp(0.5 * ln(N))).")
print(f"      GNFS is O(exp(c * (ln N)^(1/3) * (ln ln N)^(2/3))).")
if alpha_modsym >= 0.5:
    print(f"      => ModSym is SLOWER than trial division in N-scaling.")
    print(f"         For this to be useful, a shortcut must reduce the exponent.")